In [1]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import PowerTransformer
import math
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from scipy import stats
import matplotlib.style as style
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import norm, skew #for some statistics
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import GridSearchCV
# 處理 data 套件
import numpy as np
import pandas as pd
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,KFold
from scipy.stats import skew

In [2]:
def skew_pro(data):
    columns=data.drop(['Predict'], axis=1).columns
    for col in columns: 
        if abs(data[col].skew()) >= 0.7: 
            pt = PowerTransformer() 
            d = pt.fit_transform(data[col].values.reshape(-1, 1)).flatten()
            data[col]=d
    X = data[columns]
    scaler = RobustScaler()
    data[columns] = scaler.fit_transform(X)
    
    return data

def lr_rmse_ave(x,y):
    
    
    train_rmse=[]
    test_rmse=[]
    test_r2=[]
    
    for i in np.arange(10):
        x_train,x_test,y_train,y_test = train_test_split(x,y, random_state = i)
        lr = LinearRegression().fit(x_train,y_train)

        y_train_pred = lr.predict(x_train)
        y_test_pred = lr.predict(x_test)
        train_rmse.append(mean_squared_error(y_train, y_train_pred,squared=False))
        test_rmse.append(mean_squared_error(y_test, y_test_pred,squared=False))
        test_r2.append(r2_score(y_test,y_test_pred))
    
    train_rmse=np.array(train_rmse).mean()
    test_rmse=np.array(test_rmse).mean()
    test_r2=np.array(test_r2).mean()
    
#     print('train_rmse:', train_rmse)
#     print('test_rmse:', test_rmse)
#     print('test_r2:', test_r2)
    return train_rmse, test_rmse, test_r2
    

In [3]:
#
n='Input_C_013'  #改路徑
#
dire= ['./BayesianRidge_Pre_0/','./BayesianRidge_Pre_1/','./BayesianRidge_Pre_2/','./BayesianRidge_Pre_3/','./BayesianRidge_Pre_4/','./BayesianRidge_Pre_5/','./BayesianRidge_Pre_6/','./BayesianRidge_Pre_7/']
# dire= ['../BayesianRidge_Pre_0/']
print(n)
for d in dire:
    f_train_rmse=[]
    f_test_rmse=[]
    f_test_r2=[]
    print(d)
    data_d= d+n+'.csv'
    data= pd.read_csv(data_d,index_col=0).drop(['Number'], axis=1)
    y=data.Predict
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    f20=abs(numcorr['Predict']).sort_values(ascending=False).head(21).to_frame().index.to_numpy()[1:]
    fea_list=[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
    for i in fea_list:
        print('feature number:',i)
        x=skew_data[f20[:i]]
        tr_r,te_r, te_r2= lr_rmse_ave(x,y)
        print('train_rmse:',tr_r)
        print('test_rmse:',te_r)
        print('test_r2:',te_r2)
        f_train_rmse.append(tr_r)
        f_test_rmse.append(te_r)
        f_test_r2.append(te_r2)

    test_r=np.array(f_test_rmse)
    test_r2=np.array(f_test_r2)
    test_r_min= np.where(test_r == np.amin(test_r))
    test_r2_max= np.where(test_r2 == np.amax(test_r2))    
    print('Test_rmse_min = 1.fea_num:', fea_list[test_r_min[0][0]], ' 2.rmse:' ,np.amin(test_r))
    print('Test_r2_max = 1. fea_num:', fea_list[test_r2_max[0][0]],  ' 2.r2:', np.amax(test_r2))



Input_C_013
./BayesianRidge_Pre_0/
feature number: 2
train_rmse: 0.0006174413730543459
test_rmse: 0.0006353453694414728
test_r2: 0.056711762688774495
feature number: 3
train_rmse: 0.0006168713247711479
test_rmse: 0.0006362426562282719
test_r2: 0.0539646312289364
feature number: 4
train_rmse: 0.0006150542890904689
test_rmse: 0.0006373734583264556
test_r2: 0.050891450240710935
feature number: 5
train_rmse: 0.0006136686643773623
test_rmse: 0.0006389228779838784
test_r2: 0.045988380828949
feature number: 6
train_rmse: 0.0006098727775124768
test_rmse: 0.0006382861840337134
test_r2: 0.04812972267367395
feature number: 7
train_rmse: 0.0006079085727526282
test_rmse: 0.0006391145899970497
test_r2: 0.045273623490658096
feature number: 8
train_rmse: 0.0006070010869203448
test_rmse: 0.0006420380713462082
test_r2: 0.03656566323337133
feature number: 9
train_rmse: 0.0006058205459838959
test_rmse: 0.0006452945772631895
test_r2: 0.026326500903156292
feature number: 10
train_rmse: 0.0006055330110563282

train_rmse: 0.0005893205225663971
test_rmse: 0.0006427026242216684
test_r2: 0.030454279327222333
feature number: 15
train_rmse: 0.0005845004197817809
test_rmse: 0.0006361528428142627
test_r2: 0.04898672419107321
feature number: 16
train_rmse: 0.0005841485248253645
test_rmse: 0.0006384698088059375
test_r2: 0.04230588582136846
feature number: 17
train_rmse: 0.0005836240616965579
test_rmse: 0.0006403198747713041
test_r2: 0.03690622819460776
feature number: 18
train_rmse: 0.0005830443185574188
test_rmse: 0.0006417008047499651
test_r2: 0.03286968455342515
feature number: 19
train_rmse: 0.0005820001547209299
test_rmse: 0.0006435037288923725
test_r2: 0.027013147868650788
feature number: 20
train_rmse: 0.000581757593956847
test_rmse: 0.0006444959236214005
test_r2: 0.023664831101710315
Test_rmse_min = 1.fea_num: 2  2.rmse: 0.0006353268215037666
Test_r2_max = 1. fea_num: 2  2.r2: 0.056766920511065334
./BayesianRidge_Pre_4/
feature number: 2
train_rmse: 0.0006175374904899731
test_rmse: 0.00063542

train_rmse: 0.0006098756890366623
test_rmse: 0.0006382602431493599
test_r2: 0.048209336373703435
feature number: 7
train_rmse: 0.0006078970731671033
test_rmse: 0.0006389680704998544
test_r2: 0.04569115671548387
feature number: 8
train_rmse: 0.0006069883179502166
test_rmse: 0.0006418958102614701
test_r2: 0.03697518652584168
feature number: 9
train_rmse: 0.0006058525411487504
test_rmse: 0.0006451524907407474
test_r2: 0.02674550335521425
feature number: 10
train_rmse: 0.0006055646974132944
test_rmse: 0.0006468779704652994
test_r2: 0.021205882631025075
feature number: 11
train_rmse: 0.0005959041363001099
test_rmse: 0.0006395497903757836
test_r2: 0.040942652643215706
feature number: 12
train_rmse: 0.000595825777203202
test_rmse: 0.0006398043195707367
test_r2: 0.04019505757966729
feature number: 13
train_rmse: 0.000595226973260804
test_rmse: 0.0006422774551147075
test_r2: 0.032997120563980596
feature number: 14
train_rmse: 0.0005894194067012741
test_rmse: 0.0006426040008350317
test_r2: 0.030

In [4]:
fea_number=6
#
data=pd.read_csv('./BayesianRidge_Pre_0/Input_C_013.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑
#
y=data.Predict
skew_data=skew_pro(data)

num=skew_data.select_dtypes(exclude='object')
numcorr=num.corr()
title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea_number+1).to_frame().index.to_numpy()[1:]
skew_x=skew_data.drop(['Predict'], axis=1)
title

array(['Input_C_026_Y', 'Input_C_028_Y', 'Input_C_016_Y', 'Input_C_030_Y',
       'Input_C_029_Y', 'Input_C_135'], dtype=object)

## Linear Regression Baseline

In [6]:
def lr_rmse_ave_fea(data,fea_num):
    
    num=data.select_dtypes(exclude='object')
    numcorr=num.corr()
    cols=abs(numcorr['Predict']).sort_values(ascending=False).head(fea_num+1).to_frame().index.to_numpy()[1:]
    y = data['Predict']
    X= data[cols]
    train_rmse=[]
    test_rmse=[]
    test_r2=[]
    
    for i in np.arange(10):
        x_train,x_test,y_train,y_test = train_test_split(X,y, random_state = i)
        lr = LinearRegression().fit(x_train,y_train)

        y_train_pred = lr.predict(x_train)
        y_test_pred = lr.predict(x_test)
        train_rmse.append(mean_squared_error(y_train, y_train_pred,squared=False))
        test_rmse.append(mean_squared_error(y_test, y_test_pred,squared=False))
        test_r2.append(r2_score(y_test,y_test_pred))
    
    train_rmse=np.array(train_rmse).mean()
    test_rmse=np.array(test_rmse).mean()
    test_r2=np.array(test_r2).mean()
    
    print('train_rmse:', train_rmse)
    print('test_rmse:', test_rmse)
    print('test_r2:', test_r2)
#     return train_rmse, test_rmse, test_r2

In [7]:
lr_rmse_ave_fea(skew_data,6)

train_rmse: 0.0006098727775124768
test_rmse: 0.0006382861840337134
test_r2: 0.04812972267367395


## Lasso

In [8]:
def laso_alpha(x,y,parameters):
    from sklearn.linear_model import Lasso
    
    para=[]

    for i in np.arange(10):
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i)
        lasso=Lasso()
        lasso_reg=GridSearchCV(lasso, param_grid=parameters, scoring='neg_mean_squared_error', cv=3)
        lasso_reg.fit(X_train,y_train)
        para.append(lasso_reg.best_params_)

    print(para)
    
def laso_rmse_ave(x,y,alp):
    from sklearn.linear_model import Lasso
    
    rmse=[]
    r2=[]

    for i in np.arange(10):
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i)
        lasso_mod=Lasso(alpha=alp)
        lasso_mod.fit(X_train,y_train)
        y_lasso_train=lasso_mod.predict(X_train)
        y_lasso_test=lasso_mod.predict(X_test)
        rmse.append(math.sqrt(mean_squared_error(y_test, y_lasso_test)))
        r2.append(r2_score(y_test, y_lasso_test))
    
    test_rmse=np.array(rmse).mean()
    print('test_rmse_ave:',test_rmse)
    print(rmse)
    print('\n')
    test_r2=np.array(r2).mean()
    print('test_r2_ave:',test_r2)
    print(r2)
    

In [9]:
parameters= {'alpha':[0.003,0.001,0.0009,0.00009,9e-06,9e-07]}
laso_alpha(skew_x,y,parameters)

[{'alpha': 9e-05}, {'alpha': 9e-05}, {'alpha': 0.003}, {'alpha': 9e-05}, {'alpha': 9e-05}, {'alpha': 9e-05}, {'alpha': 9e-05}, {'alpha': 9e-05}, {'alpha': 9e-05}, {'alpha': 9e-05}]


In [10]:
laso_rmse_ave(skew_x,y,9e-05)

test_rmse_ave: 0.0006402738662414523
[0.0005070720603141039, 0.0005533932034008936, 0.0006658913075219401, 0.0006620476027704236, 0.0005979507367114452, 0.0006412400748036581, 0.0007203569708188521, 0.0006494430295326798, 0.0007647429461947931, 0.0006406007303457344]


test_r2_ave: 0.010919684838533283
[-0.005055990136302091, 0.009491940983552105, 0.0013808503425535301, 0.011941400616249598, 0.038264326155801553, 0.0022069131211364112, -0.0018452317535340157, 0.006885895338539227, 0.011227246148405645, 0.03469949756893087]


In [11]:
parameters= {'alpha':[0.003,0.001,0.0009,0.00009,9e-06,9e-07]}
laso_alpha(skew_data[title],y,parameters)

[{'alpha': 9e-06}, {'alpha': 9e-06}, {'alpha': 9e-06}, {'alpha': 9e-06}, {'alpha': 9e-06}, {'alpha': 9e-06}, {'alpha': 9e-06}, {'alpha': 9e-06}, {'alpha': 9e-06}, {'alpha': 9e-07}]


In [12]:
laso_rmse_ave(skew_data[title],y,9e-06)

test_rmse_ave: 0.0006283247368746782
[0.0005213062870335975, 0.0005158321773938128, 0.0006580394945820789, 0.0006723433052771762, 0.0005868094010316847, 0.0006074080626218921, 0.0007063666851048812, 0.000632535794402741, 0.0007466082714303213, 0.0006359978898685955]


test_r2_ave: 0.04620633182285675
[-0.06227465186576109, 0.13938832978730786, 0.024792306630814553, -0.01902875068138843, 0.0737695788612639, 0.1047171130704384, 0.03669120498545686, 0.05792117208379621, 0.057565610753029905, 0.04852140460360932]


## ElasticNet

In [13]:
def score(y_pred):
    return str(math.sqrt(mean_squared_error(y_test, y_pred)))

def ElasticNet_alpha(x,y,alphas,l1ratio):
    from sklearn.linear_model import ElasticNetCV
    
    alpha_=[]
    l1_ratio_=[]

    for i in np.arange(10):
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i)
        elastic_cv = ElasticNetCV(cv=3, max_iter=1e7, alphas=alphas,  l1_ratio=l1ratio)
        elasticmod = elastic_cv.fit(X_train, y_train.ravel())

        alpha_.append(elastic_cv.alpha_)
        l1_ratio_.append(elastic_cv.l1_ratio_)

    print('alpha')
    print(alpha_)
    print('\n')
    print('l1_ratio')
    print(l1_ratio_)
    
def ElasticNet_rmse_ave(x,y,alp,l1r):
    from sklearn.linear_model import ElasticNet
    
    rmse=[]
    r2=[]

    for i in np.arange(10):
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i)
        ElasticNet_mod=ElasticNet(alpha=alp, l1_ratio=l1r)
        ElasticNet_mod.fit(X_train,y_train)
        y_ElasticNet_train=ElasticNet_mod.predict(X_train)
        y_ElasticNet_test=ElasticNet_mod.predict(X_test)
        rmse.append(math.sqrt(mean_squared_error(y_test, y_ElasticNet_test)))
        r2.append(r2_score(y_test, y_ElasticNet_test))
    
    test_rmse=np.array(rmse).mean()
    print('test_rmse_ave:',test_rmse)
    print(rmse)
    print('\n')
    test_r2=np.array(r2).mean()
    print('test_r2_ave:',test_r2)
    print(r2)
    

In [15]:
alphas = [0.005]

# alphas =[0.07,0.06,0.05,0.04, 0.03,0.02,0.01,0.005,0]
l1ratio = [0.02,0.01,0.03,0.04,0.06,0.07]
ElasticNet_alpha(skew_x,y,alphas,l1ratio)

alpha
[0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005]


l1_ratio
[0.01, 0.02, 0.04, 0.01, 0.03, 0.01, 0.01, 0.02, 0.01, 0.03]


In [16]:
ElasticNet_rmse_ave(skew_x,y,[0.005],0.02)

test_rmse_ave: 0.0006407486014619978
[0.0005049945755097591, 0.0005540884934433307, 0.0006674694411158813, 0.0006619066899006291, 0.0005991588034697477, 0.0006416086748539438, 0.0007215556266691146, 0.0006489950872761998, 0.0007649727562630983, 0.0006427358661182736]


test_r2_ave: 0.009656187181818977
[0.0031626101132387063, 0.007001404324623706, -0.0033581269701545846, 0.012361960573047837, 0.03437432490271075, 0.0010594730197066582, -0.005182096123675706, 0.008255389924606726, 0.010632891987325865, 0.0282540400667598]


In [17]:
alphas = [0.005]

# alphas =[0.07,0.06,0.05,0.04, 0.03,0.02,0.01,0.005,0]
l1ratio = [0.02,0.01,0.03,0.04,0.06,0.07]
ElasticNet_alpha(skew_data[title],y,alphas,l1ratio)

alpha
[0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005]


l1_ratio
[0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]


In [18]:
ElasticNet_rmse_ave(skew_data[title],y,[0.005],0.01)

test_rmse_ave: 0.0006307443069236679
[0.0005111529665969808, 0.0005288558135568925, 0.0006615287833108095, 0.000664486562503285, 0.0005900330978057142, 0.0006159968989378111, 0.000712282227842074, 0.0006336057082614228, 0.0007545955037459572, 0.0006349055066757318]


test_r2_ave: 0.04002400642713677
[-0.021298430539504754, 0.09538259988570341, 0.014422707379912025, 0.004648045876172291, 0.06356494461804874, 0.07921925249528061, 0.020488980873268114, 0.05473148494021429, 0.03729338213186051, 0.051787096610412475]


## SVR

In [19]:
from sklearn.svm import SVR

In [21]:
# parameters = {'kernel': ('linear', 'rbf','poly'), 'C':[1.2,1.3,1,1.5],'gamma': [1e-7,1e-6 1e-8,5e-7],'epsilon':[0.2,0.5,0.3,0.4]}
for i in np.arange(10):
    X_SVR_train, X_SVR_test, y_SVR_train, y_SVR_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=i)
    parameters = {'kernel': ['rbf'], 'C':[1.2,1.3,1.1,1.4,1],'gamma': [1e-7,5e-6,5e-7],'epsilon':[0.2,0.1,0.05,0]}
    clf = GridSearchCV(estimator = SVR(), param_grid = parameters , n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    clf.fit(X_SVR_train,y_SVR_train)
    print('best params')
    print (clf.best_params_)
    print('\n')

best params
{'C': 1.1, 'epsilon': 0, 'gamma': 5e-07, 'kernel': 'rbf'}


best params
{'C': 1.4, 'epsilon': 0, 'gamma': 5e-06, 'kernel': 'rbf'}


best params
{'C': 1, 'epsilon': 0, 'gamma': 5e-06, 'kernel': 'rbf'}


best params
{'C': 1.3, 'epsilon': 0, 'gamma': 5e-06, 'kernel': 'rbf'}


best params
{'C': 1.3, 'epsilon': 0, 'gamma': 5e-06, 'kernel': 'rbf'}


best params
{'C': 1.3, 'epsilon': 0, 'gamma': 5e-06, 'kernel': 'rbf'}


best params
{'C': 1.2, 'epsilon': 0, 'gamma': 5e-06, 'kernel': 'rbf'}


best params
{'C': 1.4, 'epsilon': 0, 'gamma': 5e-06, 'kernel': 'rbf'}


best params
{'C': 1.4, 'epsilon': 0, 'gamma': 5e-06, 'kernel': 'rbf'}


best params
{'C': 1.3, 'epsilon': 0, 'gamma': 5e-06, 'kernel': 'rbf'}




In [25]:
# parameters = {'kernel': ('linear', 'rbf','poly'), 'C':[1.2,1.3,1,1.5],'gamma': [1e-7,1e-6 1e-8,5e-7],'epsilon':[0.2,0.5,0.3,0.4]}
for i in np.arange(10):
    X_SVR_train, X_SVR_test, y_SVR_train, y_SVR_test = train_test_split(skew_x, y, test_size=0.2, random_state=i)
    parameters = {'kernel': ['poly'], 'C':[1.6,1.5,1.8,1.9,2],'gamma': [1e-3,1e-5,1e-4,1e-2],'epsilon':[0.1,0.05,0,0.2]}
    clf = GridSearchCV(estimator = SVR(), param_grid = parameters , n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    clf.fit(X_SVR_train,y_SVR_train)
    print('best params')
    print (clf.best_params_)
    print('\n')

best params
{'C': 1.6, 'epsilon': 0, 'gamma': 0.001, 'kernel': 'poly'}


best params
{'C': 1.6, 'epsilon': 0, 'gamma': 0.01, 'kernel': 'poly'}


best params
{'C': 1.5, 'epsilon': 0, 'gamma': 0.001, 'kernel': 'poly'}


best params
{'C': 1.6, 'epsilon': 0, 'gamma': 0.01, 'kernel': 'poly'}


best params
{'C': 1.6, 'epsilon': 0, 'gamma': 0.01, 'kernel': 'poly'}


best params
{'C': 1.6, 'epsilon': 0, 'gamma': 0.01, 'kernel': 'poly'}


best params
{'C': 1.6, 'epsilon': 0, 'gamma': 0.01, 'kernel': 'poly'}


best params
{'C': 1.6, 'epsilon': 0, 'gamma': 0.01, 'kernel': 'poly'}


best params
{'C': 1.6, 'epsilon': 0, 'gamma': 0.001, 'kernel': 'poly'}


best params
{'C': 1.6, 'epsilon': 0, 'gamma': 0.001, 'kernel': 'poly'}




In [22]:
#best params {'C': 1.3, 'epsilon': 0.4, 'gamma': 1e-07, 'kernel': 'linear'}
def svr_rmse_ave(data,fea_num,model):
    y = data['Predict']
    if fea_num != None:
        num=data.select_dtypes(exclude='object')
        numcorr=num.corr()
        title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea_num+1).to_frame().index.to_numpy()[1:]
        X= data[title]
    else:
        X= data.drop(['Predict'], axis=1)
    
    train_rmse=[]
    test_rmse=[]
    test_r2=[]
    
    for i in np.arange(10):
        x_train,x_test,y_train,y_test = train_test_split(X,y, random_state = i)
        svr = model.fit(x_train,y_train)
        y_train_pred = svr.predict(x_train)
        y_test_pred = svr.predict(x_test)
        train_rmse.append(mean_squared_error(y_train, y_train_pred,squared=False))
        test_rmse.append(mean_squared_error(y_test, y_test_pred,squared=False))
        test_r2.append(r2_score(y_test,y_test_pred))
    
    train_rmse=np.array(train_rmse).mean()
    test_rmse=np.array(test_rmse).mean()
    test_r2=np.array(test_r2).mean()
    
    print('train_rmse:', train_rmse)
    print('test_rmse:', test_rmse)
    print('test_r2:', test_r2)
#     return train_rmse, test_rmse, test_r2

In [23]:
model=SVR(kernel='rbf', C=1.3, gamma= 5e-06, epsilon= 0)
svr_rmse_ave(skew_data,16,model)

train_rmse: 0.0006228458321759897
test_rmse: 0.0006503265771944317
test_r2: 0.013592572976759022


In [26]:
model=SVR(kernel='poly', C=1.6, gamma= 0.005, epsilon= 0)
svr_rmse_ave(skew_data,None,model)

train_rmse: 0.000194261673399986
test_rmse: 0.000654146133994295
test_r2: 0.0016895735545348733


## XGB

In [27]:
def xgb_ave(x,y,model):
    
    rmse=[]
    r2=[]

    for i in np.arange(10):
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i)
        model.fit(X_train,y_train)
        y_pred = model.predict(X_test)
        rmse.append(math.sqrt(mean_squared_error(y_test, y_pred )))
        r2.append(r2_score(y_test, y_pred ))
    
    test_rmse=np.array(rmse).mean()
    print('test_rmse_ave:',test_rmse)
    print(rmse)
    print('\n')
    test_r2=np.array(r2).mean()
    print('test_r2_ave:',test_r2)
    print(r2)
    

### 先調 booster / n_estimators

In [46]:
#for tuning parameters
parameters_for_testing = {
#     'booster':['gbtree','gblinear','dart'],
    'booster':['dart'],
    'n_estimators':range(10,201,10),
}

other_params = {'learning_rate': 0.1, 'n_estimators': 90, 'max_depth': 5, 'min_child_weight': 1, 'seed': 42,
'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1}

for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')


[04:02:34] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster': 'dart', 'n_estimators': 60}


[04:02:38] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster': 'dart', 'n_estimators': 60}


[04:02:43] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster': 'dart', 'n_estimators': 60}


[04:02:48] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster': 'dart', 'n_estimators': 70}


[04:02:53] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster': 'dart', 'n_estimators': 60}


[04:02:58] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster': 'dart', 'n

In [29]:
#for tuning parameters
parameters_for_testing = {
#     'booster':['gbtree','gblinear','dart'],
    'booster':['gbtree'],
#     'n_estimators':[50,60,70],
    'n_estimators':range(60,221,10),
}

other_params = {'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 1, 'seed': 42,
'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1}

for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_x, y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')


[02:13:59] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster': 'gbtree', 'n_estimators': 90}


[02:14:09] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster': 'gbtree', 'n_estimators': 90}


[02:14:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster': 'gbtree', 'n_estimators': 90}


[02:14:26] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster': 'gbtree', 'n_estimators': 150}


[02:14:35] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster': 'gbtree', 'n_estimators': 190}


[02:14:44] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster'

### 調 max_depth / min_child_weight

In [48]:
# {'booster': 'dart', 'n_estimators': 60}

parameters_for_testing = {
#     'max_depth': [1,2,3, 4, 5, 6, 7, 8, 9, 10],
    'max_depth': [2],
    'min_child_weight': [1, 2, 3, 4, 5, 6]
}

other_params = {'booster':'dart','learning_rate': 0.1, 'n_estimators': 60, 'seed': 42,
'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1}

for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')

[04:04:56] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 2, 'min_child_weight': 2}


[04:04:57] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 2, 'min_child_weight': 1}


[04:04:57] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 2, 'min_child_weight': 1}


[04:04:57] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 2, 'min_child_weight': 4}


[04:04:58] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 2, 'min_child_weight': 1}


[04:04:58] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 2, 'min_

In [31]:
# {'booster': 'dart', 'n_estimators': 160}

parameters_for_testing = {
#     'max_depth': [1,2,3, 4, 5, 6, 7, 8, 9, 10],
    'max_depth': [5],
    'min_child_weight': [1, 2, 3, 4, 5, 6]
}

other_params = {'booster':'gbtree','learning_rate': 0.1, 'n_estimators': 110, 'seed': 42,
'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1}

for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_x, y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')

[02:23:42] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 5, 'min_child_weight': 6}


[02:23:45] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 5, 'min_child_weight': 5}


[02:23:47] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 5, 'min_child_weight': 3}


[02:23:49] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 5, 'min_child_weight': 3}


[02:23:53] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 5, 'min_child_weight': 6}


[02:23:55] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 5, 'min_

### 調 gamma

In [49]:
# {'max_depth': 2, 'min_child_weight': 2}
parameters_for_testing = {
    'gamma': [0,0.1, 0.2, 0.3, 0.4, 0.5, 0.6]
}

other_params = {'booster':'dart','learning_rate': 0.1, 'n_estimators': 60, 'max_depth': 2, 'min_child_weight': 2, 'seed': 42,
'subsample': 0.8, 'colsample_bytree': 0.8, 'reg_alpha': 0, 'reg_lambda': 1}

for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')

[04:05:43] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}


[04:05:44] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}


[04:05:44] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}


[04:05:45] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}


[04:05:45] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}


[04:05:46] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}


[04:05:46] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}



In [32]:
# {'max_depth': 7, 'min_child_weight': 3}
parameters_for_testing = {
    'gamma': [0,0.1, 0.2, 0.3, 0.4, 0.5, 0.6]
}

other_params = {'booster':'gbtree','learning_rate': 0.1, 'n_estimators': 110, 'max_depth': 5, 'min_child_weight': 4, 'seed': 42,
'subsample': 0.8, 'colsample_bytree': 0.8, 'reg_alpha': 0, 'reg_lambda': 1}

for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_x, y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')

[02:24:59] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0.1}


[02:25:06] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0.1}


[02:25:10] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}


[02:25:15] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}


[02:25:20] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0.1}


[02:25:24] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0.1}


[02:25:29] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamm

### 調 subsample / colsample_bytree

In [51]:
# {'gamma': 0}
parameters_for_testing = {
#     'colsample_bytree': [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
    'colsample_bytree': [0.5],
    'subsample': [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9] 
}

other_params = {'booster':'dart','learning_rate': 0.1, 'n_estimators': 60, 'max_depth': 2, 'min_child_weight': 2, 'seed': 42,
 'reg_alpha': 0, 'reg_lambda': 1, 'gamma':0}

for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')

[04:09:16] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'colsample_bytree': 0.5, 'subsample': 0.3}


[04:09:16] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'colsample_bytree': 0.5, 'subsample': 0.7}


[04:09:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'colsample_bytree': 0.5, 'subsample': 0.7}


[04:09:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'colsample_bytree': 0.5, 'subsample': 0.7}


[04:09:18] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'colsample_bytree': 0.5, 'subsample': 0.9}


[04:09:18] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'

In [33]:
# {'gamma': 0}
parameters_for_testing = {
    'colsample_bytree': [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
#     'colsample_bytree': [0.6],
    'subsample': [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9] 
}

other_params = {'booster':'gbtree','learning_rate': 0.1, 'n_estimators': 110, 'max_depth': 5, 'min_child_weight': 4, 'seed': 42,
 'reg_alpha': 0, 'reg_lambda': 1, 'gamma':0.05}

for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_x, y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')

[02:26:26] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'colsample_bytree': 0.2, 'subsample': 0.9}


[02:26:54] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'colsample_bytree': 0.2, 'subsample': 0.3}


[02:27:22] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'colsample_bytree': 0.2, 'subsample': 0.2}


[02:27:50] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'colsample_bytree': 0.2, 'subsample': 0.5}


[02:28:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'colsample_bytree': 0.2, 'subsample': 0.7}


[02:28:45] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'

### reg_alpha / reg_lambda

In [52]:
# {'colsample_bytree': 0.5, 'subsample': 0.6}

parameters_for_testing = {
    'reg_alpha': [0,0.05, 0.1, 0.5 , 1, 2, 3], 
#     'reg_alpha': [0.5],     
    'reg_lambda': [0.05, 0.1, 0.5 , 1, 2, 3]
}

other_params = {'booster':'dart','learning_rate': 0.1, 'n_estimators': 60, 'max_depth': 2, 'min_child_weight': 2, 'seed': 42,
 'gamma':0,'subsample':0.6,'colsample_bytree': 0.5}


for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')

[04:10:13] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0, 'reg_lambda': 1}


[04:10:15] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0, 'reg_lambda': 0.5}


[04:10:18] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0, 'reg_lambda': 0.5}


[04:10:20] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0, 'reg_lambda': 0.5}


[04:10:22] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0, 'reg_lambda': 1}


[04:10:25] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0, 'reg_lambda': 1}


[04:10:27]

In [34]:
# {'colsample_bytree': 0.9, 'subsample': 0.7}

parameters_for_testing = {
    'reg_alpha': [0,0.05, 0.1, 0.5 , 1, 2, 3], 
#     'reg_alpha': [0.02],     
    'reg_lambda': [0.05, 0.1, 0.5 , 1, 2, 3]
}

other_params = {'booster':'gbtree','learning_rate': 0.1, 'n_estimators': 110, 'max_depth': 5, 'min_child_weight': 4, 'seed': 42,
 'gamma':0.05,'subsample':0.6,'colsample_bytree': 0.2}


for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_x, y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')

[02:32:08] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0, 'reg_lambda': 3}


[02:32:16] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0, 'reg_lambda': 3}


[02:32:24] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0, 'reg_lambda': 0.1}


[02:32:32] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0, 'reg_lambda': 1}


[02:32:40] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0, 'reg_lambda': 3}


[02:32:48] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0, 'reg_lambda': 0.05}


[02:32:56] 

### learning rate

In [53]:
# {'reg_alpha': 0, 'reg_lambda': 0.7}

parameters_for_testing = {
   'learning_rate': [0.01, 0.05, 0.07, 0.1, 0.2]
}

other_params = {'booster':'dart', 'n_estimators': 60, 'max_depth': 2, 'min_child_weight': 2, 'seed': 42,
 'gamma':0,'subsample':0.6,'colsample_bytree': 0.5,'reg_alpha': 0, 'reg_lambda': 0.7}

for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')

[04:11:43] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.1}


[04:11:43] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.1}


[04:11:44] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.1}


[04:11:44] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.1}


[04:11:44] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.1}


[04:11:45] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.2}


[04:11:45] WARNING: src/objective/regression_obj.cu:152: reg:linear is now depreca

In [35]:
# {'reg_alpha': 0.0, 'reg_lambda': 0.3}

parameters_for_testing = {
   'learning_rate': [0.01, 0.05, 0.07, 0.1, 0.2]
}

other_params = {'booster':'gbtree', 'n_estimators': 110, 'max_depth': 5, 'min_child_weight': 4, 'seed': 42,
 'gamma':0.05,'subsample':0.6,'colsample_bytree': 0.2,'reg_alpha': 0, 'reg_lambda': 1}

for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_x, y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')

[02:34:09] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.1}


[02:34:10] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.1}


[02:34:11] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.2}


[02:34:14] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.2}


[02:34:15] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.1}


[02:34:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.2}


[02:34:18] WARNING: src/objective/regression_obj.cu:152: reg:linear is now depreca

In [131]:
best_xgb_model = XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.1,
              booster = 'dart', 
              n_estimators = 60, 
              max_depth = 2, 
              min_child_weight = 2,
              seed = 42,
              gamma = 0,
              subsample = 0.85,
              colsample_bytree = 0.4,
              reg_alpha =  0,
              reg_lambda = 0.7)
xgb_ave(skew_data[title], y,best_xgb_model)

test_rmse_ave: 0.008496031540705907
[0.007648650629856374, 0.007861428699795805, 0.008923526006532567, 0.008422016710058338, 0.007514787893473913, 0.007671313900216453, 0.009908224384169528, 0.0094945394022681, 0.008777706642056635, 0.008738121138631347]


test_r2_ave: 0.5658204873179591
[0.6994093043442213, 0.6676009790074625, 0.49424336268772506, 0.5730727307503245, 0.6729951732468282, 0.6589820348516682, 0.3470225948052349, 0.46909149956225205, 0.5156770986673376, 0.5601100952565365]


In [36]:
best_xgb_model = XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.15,
              booster = 'gbtree', 
              n_estimators = 110, 
              max_depth = 5, 
              min_child_weight = 4,
              seed = 42,
              gamma = 0.05,
              subsample = 0.6,
              colsample_bytree = 0.2,
              reg_alpha =  0,
              reg_lambda = 1)
xgb_ave(skew_x, y,best_xgb_model)


test_rmse_ave: 0.000648536563902077
[0.00050899781611256, 0.0005562524087282393, 0.000677831078391039, 0.0006661289796088438, 0.000611026549346121, 0.0006491606772872192, 0.0007330537271913988, 0.0006563156746547682, 0.0007744748995162602, 0.0006521238281843198]


test_r2_ave: -0.014151297207583392
[-0.012704479786386447, -0.0007697756047722049, -0.034751710565168015, -0.0002784749039481671, -0.004257484130929523, -0.022594818491489033, -0.03747280553331267, -0.014244317854156696, -0.014098697613002198, -0.0003404075926689565]


## RandomForest

In [37]:
def rand_ave(x,y,model):
    
    rmse=[]
    r2=[]

    for i in np.arange(10):
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i)
        model.fit(X_train,y_train)
        y_pred = model.predict(X_test)
        rmse.append(math.sqrt(mean_squared_error(y_test, y_pred )))
        r2.append(r2_score(y_test, y_pred ))
    
    test_rmse=np.array(rmse).mean()
    print('test_rmse_ave:',test_rmse)
    print(rmse)
    print('\n')
    test_r2=np.array(r2).mean()
    print('test_r2_ave:',test_r2)
    print(r2)
    

In [78]:
from sklearn.ensemble import RandomForestRegressor
clf = RandomForestRegressor(
          criterion ='mse', 
          random_state = 42
)

In [79]:
X_RAND_train, X_RAND_test, y_RAND_train, y_RAND_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=42)

In [81]:
param_grid = {
#               'n_estimators': [50,100,200 ,500, 900],
              'n_estimators': [20,40,50,60,80],
              'max_depth': range(1,10,1),
              'min_samples_split': range(2,10,1)}

gsearch = GridSearchCV(estimator = clf, param_grid = param_grid, n_jobs=6,iid=False, verbose=10,scoring='neg_mean_squared_error')

gsearch.fit(X_RAND_train,y_RAND_train)

print('best params')
print (gsearch.best_params_)
print('best score')
print (gsearch.best_score_)

Fitting 5 folds for each of 360 candidates, totalling 1800 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Batch computation too fast (0.0688s.) Setting batch_size=2.
[Parallel(n_jobs=6)]: Done   6 tasks      | elapsed:    0.2s
[Parallel(n_jobs=6)]: Done  14 tasks      | elapsed:    0.8s
[Parallel(n_jobs=6)]: Done  28 tasks      | elapsed:    1.1s
[Parallel(n_jobs=6)]: Done  46 tasks      | elapsed:    1.8s
[Parallel(n_jobs=6)]: Done  64 tasks      | elapsed:    2.3s
[Parallel(n_jobs=6)]: Done  86 tasks      | elapsed:    3.1s
[Parallel(n_jobs=6)]: Done 108 tasks      | elapsed:    3.8s
[Parallel(n_jobs=6)]: Done 134 tasks      | elapsed:    4.8s
[Parallel(n_jobs=6)]: Done 160 tasks      | elapsed:    5.9s
[Parallel(n_jobs=6)]: Done 190 tasks      | elapsed:    7.3s
[Parallel(n_jobs=6)]: Done 220 tasks      | elapsed:    8.4s
[Parallel(n_jobs=6)]: Done 254 tasks      | elapsed:    9.5s
[Parallel(n_jobs=6)]: Done 288 tasks      | elapse

best params
{'max_depth': 4, 'min_samples_split': 4, 'n_estimators': 50}
best score
-6.733198261475197e-05


[Parallel(n_jobs=6)]: Done 1800 out of 1800 | elapsed:  1.4min finished


In [82]:
# {'max_depth': 4, 'min_samples_split': 4, 'n_estimators': 50} 
param_grid = {
              "min_samples_leaf": range(1,10,1),
              "max_leaf_nodes": range(2,15,1)
              }
    
clf = RandomForestRegressor(
            criterion ='mse', 
            n_estimators = 50,
            max_depth = 4,
            min_samples_split = 4,
            random_state = 42
)

gsearch = GridSearchCV(estimator = clf, param_grid = param_grid, n_jobs=6,iid=False, verbose=10,scoring='neg_mean_squared_error')

gsearch.fit(X_RAND_train,y_RAND_train)

print('best params')
print (gsearch.best_params_)
print('best score')
print (gsearch.best_score_)

Fitting 5 folds for each of 117 candidates, totalling 585 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=6)]: Done   6 tasks      | elapsed:    0.3s
[Parallel(n_jobs=6)]: Done  13 tasks      | elapsed:    0.7s
[Parallel(n_jobs=6)]: Done  20 tasks      | elapsed:    0.9s
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:    1.2s
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    1.6s
[Parallel(n_jobs=6)]: Done  49 tasks      | elapsed:    2.1s
[Parallel(n_jobs=6)]: Done  60 tasks      | elapsed:    2.5s
[Parallel(n_jobs=6)]: Done  73 tasks      | elapsed:    3.2s
[Parallel(n_jobs=6)]: Done  86 tasks      | elapsed:    3.7s
[Parallel(n_jobs=6)]: Done 101 tasks      | elapsed:    4.2s
[Parallel(n_jobs=6)]: Done 116 tasks      | elapsed:    4.8s
[Parallel(n_jobs=6)]: Done 133 tasks      | elapsed:    5.6s
[Parallel(n_jobs=6)]: Done 150 tasks      | elapsed:    6.5s
[Parallel(n_jobs=6)]: Done 169 tasks      | elapsed:    7.6s
[Parallel(

best params
{'max_leaf_nodes': 14, 'min_samples_leaf': 2}
best score
-6.728825706399575e-05


[Parallel(n_jobs=6)]: Done 585 out of 585 | elapsed:   24.0s finished


In [83]:
rand_model = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 50,
            max_depth = 4,
            min_samples_split = 4,
            max_leaf_nodes = 14,
            min_samples_leaf =2,
            random_state = 42) 
rand_ave(skew_data[title], y,rand_model)

test_rmse_ave: 0.008359151931886303
[0.008046421179911343, 0.007522207654460264, 0.008970754200764595, 0.008192033878800358, 0.007261528804827015, 0.007389966886686526, 0.009434354784131698, 0.009095030814723662, 0.00923126977742949, 0.008447951337128053]


test_r2_ave: 0.5800138319443187
[0.6673317079609893, 0.6956681454215288, 0.4888757128331376, 0.5960708670050614, 0.6946648253743058, 0.6835371494626516, 0.40798747419915105, 0.5128303480153389, 0.46433192124209743, 0.5888401679289246]


In [38]:
from sklearn.ensemble import RandomForestRegressor
clf = RandomForestRegressor(
          criterion ='mse', 
          random_state = 42
)
X_RAND_train, X_RAND_test, y_RAND_train, y_RAND_test = train_test_split(skew_x, y, test_size=0.2, random_state=42)

In [39]:
param_grid = {
              'n_estimators': [50,40,35],
              'max_depth': range(1,10,1),
              'min_samples_split': range(2,10,1)}

gsearch = GridSearchCV(estimator = clf, param_grid = param_grid, n_jobs=6,iid=False, verbose=10,scoring='neg_mean_squared_error')

gsearch.fit(X_RAND_train,y_RAND_train)

print('best params')
print (gsearch.best_params_)
print('best score')
print (gsearch.best_score_)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 tasks      | elapsed:    0.3s
[Parallel(n_jobs=6)]: Done   6 tasks      | elapsed:    0.5s
[Parallel(n_jobs=6)]: Done  13 tasks      | elapsed:    1.2s
[Parallel(n_jobs=6)]: Done  20 tasks      | elapsed:    1.7s
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:    2.5s
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    3.2s
[Parallel(n_jobs=6)]: Done  49 tasks      | elapsed:    3.9s
[Parallel(n_jobs=6)]: Done  60 tasks      | elapsed:    4.3s
[Parallel(n_jobs=6)]: Done  73 tasks      | elapsed:    4.9s
[Parallel(n_jobs=6)]: Done  86 tasks      | elapsed:    5.5s
[Parallel(n_jobs=6)]: Done 101 tasks      | elapsed:    6.2s
[Parallel(n_jobs=6)]: Done 116 tasks      | elapsed:    6.9s
[Parallel(n_jobs=6)]: Done 133 tasks      | elapsed:    7.9s
[Parallel(n_jobs=6)]: Done 150 tasks      | elapsed:    8.8s
[Parallel(n_jobs=6)]: Done 169 tasks      | elapsed:   10.1s
[Parallel(

best params
{'max_depth': 8, 'min_samples_split': 4, 'n_estimators': 40}
best score
-4.1569960136007114e-07


In [40]:
# {'max_depth': 4, 'min_samples_split': 4, 'n_estimators': 50} 
param_grid = {
              "min_samples_leaf": range(1,10,1),
              "max_leaf_nodes": range(2,15,1)
              }
    
clf = RandomForestRegressor(
            criterion ='mse', 
            n_estimators = 40,
            max_depth = 8,
            min_samples_split = 4,
            random_state = 42
)

gsearch = GridSearchCV(estimator = clf, param_grid = param_grid, n_jobs=6,iid=False, verbose=10,scoring='neg_mean_squared_error')

gsearch.fit(X_RAND_train,y_RAND_train)

print('best params')
print (gsearch.best_params_)
print('best score')
print (gsearch.best_score_)

Fitting 5 folds for each of 117 candidates, totalling 585 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 tasks      | elapsed:    0.3s
[Parallel(n_jobs=6)]: Done   6 tasks      | elapsed:    0.4s
[Parallel(n_jobs=6)]: Done  13 tasks      | elapsed:    0.9s
[Parallel(n_jobs=6)]: Done  20 tasks      | elapsed:    1.4s
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:    2.1s
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    2.5s
[Parallel(n_jobs=6)]: Done  49 tasks      | elapsed:    3.6s
[Parallel(n_jobs=6)]: Done  60 tasks      | elapsed:    4.9s
[Parallel(n_jobs=6)]: Done  73 tasks      | elapsed:    6.5s
[Parallel(n_jobs=6)]: Done  86 tasks      | elapsed:    8.2s
[Parallel(n_jobs=6)]: Done 101 tasks      | elapsed:   10.3s
[Parallel(n_jobs=6)]: Done 116 tasks      | elapsed:   11.7s
[Parallel(n_jobs=6)]: Done 133 tasks      | elapsed:   13.2s
[Parallel(n_jobs=6)]: Done 150 tasks      | elapsed:   15.7s
[Parallel(n_jobs=6)]: Done 169 tasks      | elapsed:   18.5s
[Parallel(

best params
{'max_leaf_nodes': 12, 'min_samples_leaf': 9}
best score
-4.0542859481747725e-07


In [41]:
rand_model = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 40,
            max_depth = 8,
            min_samples_split = 4,
            max_leaf_nodes = 12,
            min_samples_leaf =9,
            random_state = 42) 
rand_ave(skew_x, y,rand_model)

test_rmse_ave: 0.0006445636291917496
[0.0005071306890571894, 0.0005731646662834049, 0.0006887186010405834, 0.0006926402224319985, 0.0006110764452493771, 0.0006280732342927883, 0.0006906207820123708, 0.0006667432761642719, 0.0007621862639336819, 0.0006252821114518297]


test_r2_ave: -0.004866801917095776
[-0.005288416967005993, -0.0625495480919338, -0.06825964790315941, -0.0814829661342884, -0.004421504434884538, 0.04276239697744533, 0.0791595359650974, -0.04672915184628046, 0.0178275084077254, 0.08031377485632674]
